# **Процесс парсинга**

Парсинг производился по сайту `metacritics` - агрегатору оценок пользователей и критиков на игры, книги, фильмы и сериалы. В данном случае, конечно, нам понадобился раздел с играми - `https://www.metacritic.com/browse/game/`.

Сайт достаточно удобен для парсинга, поскольку содержит необходимую информацию в карточках игр (в данном случае ссылки на игры, по которым мы в будущем переходим), а также разбит на страницы.

Итак, для парсинга нам понадобилось воспользоваться инструментом Selenium.

In [ ]:
import logging, sys, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import SessionNotCreatedException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

## Логирование

Был настроен лог в следующем формате: [время события] [тип события] [само сообщение]. В ходе парсинга лог печатался непосредственно в консоль.

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s", handlers=[logging.StreamHandler(sys.stdout)])
logger = logging.getLogger(__name__)

Константные переменные:

In [ ]:
START_URL_TEMPLATE = "https://www.metacritic.com/browse/game/?releaseYearMin=1958&releaseYearMax=2025&page={page}"
TOTAL_PAGES = 582
PAGE_SLEEP = 2.5

## Функция `try_start_driver()`

In [ ]:
def try_start_driver():

Здесь мы устанавливаем нужный chromedriver и сохраняем путь для сервиса Selenium, к которому он будет обращаться при выполнении парсинга.

In [ ]:
    chromedriver_path = ChromeDriverManager().install()
    service = Service(chromedriver_path)

Также настраиваются опции при запуске парсинга:

*   отключение режима песочницы Хрома
*   отключение аппаратного ускорения (для избежания ошибок рендеринга изображений, к примеру, в headless-режиме парсинга)

* фиксированный размер окна браузера
* замененный user-agent, который позволит нам обойти анти-скрейпинговую защиту сайта (**она там есть**).

Затем отправляем сообщение об успешном запуске драйвера с отображением вкладок браузера.

In [ ]:
    opts = Options()
    # Запускаем без headless, чтобы убедиться, что Chrome стартует
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-gpu")
    opts.add_argument("window-size=1200,900")
    opts.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(service=service, options=opts)
    logger.info("Успешно запущен драйвер без headless.")
    return driver

## Функция `scrape_target_products()` - сам механизм скрейпинга

Теперь перейдем непосредственно к самому механизму скрейпинга - будем собирать с каждой страницы сайта нужные нам данные (в данном случае ссылку) и будем выводить в лог последнюю карточку со страницы, на которой мы находимся, для наглядности.

В случае ошибки при парсинге также выводится соответствующее сообщение в лог с пойманным исключением.

При завершении парсинга выходим из браузера и отправляем сообщение в лог об этом.

Если нам потребуется несколько селекторов для сбора информации, то мы просто укажем их явно в коде (вместо card_selector будут другие, нужные нам) и так же уже будем ждать до момента их появления (wait_until_presence).

In [ ]:
def scrape_target_products():
    results = []
    driver = None

    def show_last_card():
        if results:
            last = results[-1]
            logger.info("Последняя карточка на этой странице: %s", last)

    try:
        driver = try_start_driver()
        from selenium.webdriver.common.by import By
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.support import expected_conditions as EC

        wait = WebDriverWait(driver, 15)
        CARD_SELECTOR = (
            "#__layout > div > div.c-layoutDefault_page > div.c-trackScroll > main > section > div.c-productListings > div > div"
        )

        for page in tqdm(range(1, TOTAL_PAGES + 1), desc="Сбор страниц", ncols=100):
            url = START_URL_TEMPLATE.format(page=page)
            driver.get(url)
            time.sleep(PAGE_SLEEP)

            with open(f"page_debug_page_{page}.html", "w", encoding="utf-8") as fh:
                fh.write(driver.page_source)

            try:
                cards = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, CARD_SELECTOR)))
            except Exception:
                cards = driver.find_elements(By.CSS_SELECTOR, CARD_SELECTOR)

            for card in cards:
                try:
                    title_elem = card.find_element(By.CSS_SELECTOR, "div.c-finderProductCard_info.u-flexbox-column > div.c-finderProductCard_title")
                    title = title_elem.text.strip()
                except Exception:
                    title = ""
                try:
                    link_elem = card.find_element(By.CSS_SELECTOR, "a")
                    link = link_elem.get_attribute("href")
                except Exception:
                    link = ""
                results.append({"page": page, "title": title, "link": link})

            show_last_card()

        logger.info("Всего собрано записей: %d", len(results))

    except Exception as e:
        logger.exception("Ошибка при парсинге: %s", e)
    finally:
        if driver:
            driver.quit()
            logger.info("Браузер закрыт.")

## Просмотр результатов

In [ ]:
    df = pd.DataFrame(results)
    df.to_csv("metacritic_results.csv", index=False, encoding="utf-8-sig")
    print(df.head())

# Merge API-данных и скрап-данных

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
parse = pd.read_csv("metacritics_edited.csv")
api = pd.read_csv("rawg_games-2.csv")

In [ ]:
del api["source"], parse["Unnamed: 0"], parse["company"]

Еще раз посмотрим на колонки наших таблиц, пропуски в них (их наличие и количество).

In [ ]:
parse.info()

In [ ]:
api.info()

In [ ]:
api.head(1)

По какой-то причине метод `.columns` странным образом менял количество колонок таблицы, поэтому приходится вот таким образом явно "прицеплять" колонку `title` к остальным. А надо было просто поменять одно название.

In [ ]:
api.columns = ["title"] + list(api.columns[1:])

Пока дозаполним пропуски в оценках и их количестве строчными нулями, потому что тип данных в этих столбцах - `object`/

In [ ]:
parse["user_score_amount"] = parse["user_score_amount"].fillna("0")
parse["metascore_amount"] = parse["metascore_amount"].fillna("0")

В столбце `user_score_amount` и `metascore_amount` данные выглядят наподобие "Reviewed by {n} users. Лишний текст необходимо убрать. Сделаем это с помощью регулярных выражений.

In [ ]:
def extract_numbers(s):
  s = s.replace(",","")
  s = re.search(r"\d+", s)
  return s.group()

parse["user_score_amount"] = parse["user_score_amount"].apply(extract_numbers)
parse["metascore_amount"] = parse["metascore_amount"].apply(extract_numbers)

In [ ]:
parse.head(3)

Теперь с помощью регулярных выражений отнормируем столбцы с названиями.

In [ ]:
parse["title"] = (
    parse["title"]
    .str.lower()                                # в нижний регистр
    .str.replace("[^a-z0-9 ]", "", regex=True)  # убираем все кроме букв/цифр/пробелов
    .str.replace("\s+", " ", regex=True)        # заменяем несколько пробелов на один
    .str.strip())                                # убираем пробелы по краям

api["title"] = (
    api["title"]
    .str.lower()                                # в нижний регистр
    .str.replace("[^a-z0-9 ]", "", regex=True)  # убираем все кроме букв/цифр/пробелов
    .str.replace("\s+", " ", regex=True)        # заменяем несколько пробелов на один
    .str.strip())                                # убираем пробелы по краям

Посмотрим на результат нормировки:

In [ ]:
parse.head(1)

In [ ]:
api.head(1)

Произведем outer-merge двух таблиц.

In [ ]:
merged = pd.merge(parse,api,how="outer",on="title")

In [ ]:
merged.info()

Теперь произведем left-merge двух таблиц. Для анализа данных он представляется нам более важным, так как данных с парсинга было собрано больше, чем данных с api.

In [ ]:
merged_left = pd.merge(parse,api,how="left",on="title")

In [ ]:
merged_left.info()

Переименуем колонки таблицы левого мерджа, чтобы было понятно, какие данные при помощи чего добыты.

In [ ]:
old_cols = list(merged_left.columns)
new_cols = []
for col in old_cols:
    new_name = col.replace("_x", "_parsed").replace("_y", "_api")
    new_cols.append(new_name)

In [ ]:
merged_left.columns = new_cols

In [ ]:
merged_left.info()

In [ ]:
merged.info()

Так же переименуем и таблицу внешнего мерджа.

In [ ]:
merged.columns = new_cols

In [ ]:
merged

In [ ]:
merged.info()

In [ ]:
merged.head(30)

Сохраним смердженные таблицы в файлы.

In [ ]:
merged_left.to_csv("merged_left.csv")
merged.to_csv("merged_outer.csv")